## Versions

Python:

pandas:

Pip:

Biopython:

keggtools:

# Retrieve EC codes from Mgx data

Install Pandas package from pip:

> python -m pip install pandas

In [2]:
import pandas as pd

Mgx = pd.read_table('../analysis/data/mgxData')

# split the column by ":" and put EC codes in Mgx_EC
Mgx_EC = Mgx["Gene.Family"].str.split(":", expand=True)[0]


# KEGG pathways from EC codes

Install Biopython package from pip:

> python -m pip install Biopython

In [77]:
from Bio.KEGG import REST
import json

pathways_dict = {}
for ec_number in Mgx_EC:
    if not ec_number.islower(): # against Bad request error from preliminary EC codes (e.g. "1.3.1.n3" -> Mgx_EC[430] )
        pathways = REST.kegg_link("pathway", ec_number).read().strip().split("\n")
        if not pathways == [""]: # against no responses to request
            for pathway in pathways:
                pathway_id, pathway_name = pathway.split("\tpath:")
                pathway_id = pathway_id.split(":")[1]
                if pathway_id not in pathways_dict:
                    pathways_dict[pathway_id] = []
                pathways_dict[pathway_id].append(pathway_name)

with open("pathways_dict.txt", "w") as file:
    json.dump(pathways_dict, file)  # encode dict into JSON

## Loading the dictionnary of pathways

In [ ]:
import json

# Open the file for reading
with open("pathways_dict.txt", "r") as file:
    # Load the dictionary from the file
    pathways_dict = json.load(file)

# Enrichment and Testing

Install keggtools package from pip:

> python -m pip install keggtools

In [30]:
from keggtools import Enrichment

# Add pathway object to list
pathway_list = records

# Init analysis with organism code
analysis = Enrichment(pathways=pathway_list)

# Study genes as list of entrez gene id's
study_genes = Mgx_EC
analysis.run_analysis(gene_list=study_genes)

# to_dataframe method requires pandas installation
result = analysis.to_dataframe()
print(result.head())

ValueError: need to 'run_analysis' first